In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

### Reading the Files.

In [2]:
# Initialize the paths
firm_rois_path = 'ROI_data.csv'
monthly_roi_path = 'roi_mat.csv'

In [3]:
# Reading the monthly ROI file.
monthly_roi = pd.read_csv(monthly_roi_path)
monthly_roi = monthly_roi.rename(columns={'Unnamed: 0': 'Months'})
display(monthly_roi)

# Reading the ROIs provided by the firms.
roi_firms = pd.read_csv(firm_rois_path)
display(roi_firms)

,Months,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
5,June,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
6,July,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
7,August,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
8,September,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4
9,October,3.0,3.0,3.1,4.6,3.1,3.3,3.2,2.30,2.5,3.2


,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


### Budget allocation for the firm ROIs.

In [4]:
# Iterating over the ROIs received by the two independent firms, i.e, over "ROI_data.csv"
roi_firms = roi_firms.drop(['Platform'], axis=1)
roi_firms_cols = list(roi_firms.columns)
roi_firms = np.array(roi_firms)
roi_firms_all_cols = ['']
for val in roi_firms_cols:
    roi_firms_all_cols.append(val)

initial_budget = int(input("Enter the total marketing budget:")) # Initialize the budget
segment_budget = int(input("Enter the individual marketing budget upper bound:"))
allocations = []
for j, rois in enumerate(roi_firms):
    row = [f'Budget Allocation for ROI {j}']
    Mod = gp.Model()
    n = len(rois)
    ModX = Mod.addMVar(n) # Initializing 10 variables for 10 marketing sources.

    # Creating a variable dictionary.
    Mod_var = {}
    for i in range(n):
        Mod_var[roi_firms_cols[i]] = ModX[i]
    
    # Creating the objective function.
    obj = 0
    for i in range(n):
        obj += (1 + rois[i])*ModX[i]
    
    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    # Adding Constraints
    # Constarint for total budget
    sum = 0
    for i in range(n):
        sum += ModX[i]
    Mod.addConstr(sum <= initial_budget) # Constarint for total budget
    Mod.addConstr(Mod_var['Print'] + Mod_var['TV'] <= Mod_var['Facebook'] + Mod_var['Email']) # Investment in Print and TV should be no more than the investment in Facebook and Email
    Mod.addConstr(Mod_var['Facebook']+ Mod_var['LinkedIn'] + Mod_var['Instagram'] + Mod_var['Snapchat'] + Mod_var['Twitter'] >= 2*(Mod_var['SEO'] + Mod_var['AdWords'])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
    for i in range(n): # Adding constraint for investment in each segment less than $ 3M
        Mod.addConstr(ModX[i] <= segment_budget)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    globals()["Allocation_"+str(j+1)] = ModX.X
    globals()["Objective_Value_"+str(j+1)] = Mod.objVal

    for val in list(ModX.X):
        row.append(val)
    allocations.append(row)

allocations = pd.DataFrame(allocations, columns=roi_firms_all_cols)
allocations

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


,,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,Budget Allocation for ROI 0,0.0,3.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,3.0
1,Budget Allocation for ROI 1,3.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0,0.0,0.0


### Calculating the difference in overall ROI based on allocations.

In [5]:
diff_1 = Objective_Value_1 - (10 + np.dot(roi_firms[0], Allocation_2))
print("Using allocations from second iteration the objective of first iteration would decrease by ", diff_1)

diff_2 = Objective_Value_1 - (10 + np.dot(roi_firms[1], Allocation_1))
print("Using allocations from first iteration the objective of second iteration would decrease by ", diff_2)

Using allocations from second iteration the objective of first iteration would decrease by  0.20399999999999885
Using allocations from first iteration the objective of second iteration would decrease by  0.19200000000000017


### Finding if third constraint is useful.

In [6]:
# Iterating over the ROIs received by the two independent firms, i.e, over "ROI_data.csv"
roi_firms = pd.read_csv(firm_rois_path)
roi_firms = roi_firms.drop(['Platform'], axis=1)
roi_firms_cols = list(roi_firms.columns)
roi_firms = np.array(roi_firms)
roi_firms_all_cols = ['']
for val in roi_firms_cols:
    roi_firms_all_cols.append(val)

initial_budget = int(input("Enter the total marketing budget:")) # Initialize the budget
allocations = []
for j, rois in enumerate(roi_firms):
    row = [f'Budget Allocation for ROI {j}']
    Mod = gp.Model()
    n = len(rois)
    ModX = Mod.addMVar(n) # Initializing 10 variables for 10 marketing sources.

    # Creating a variable dictionary.
    Mod_var = {}
    for i in range(n):
        Mod_var[roi_firms_cols[i]] = ModX[i]
    
    # Creating the objective function.
    obj = 0
    for i in range(n):
        obj += (1 + rois[i])*ModX[i]
    
    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    # Adding Constraints
    # Constarint for total budget
    sum = 0
    for i in range(n):
        sum += ModX[i]
    Mod.addConstr(sum <= initial_budget) # Constarint for total budget
    Mod.addConstr(Mod_var['Print'] + Mod_var['TV'] <= Mod_var['Facebook'] + Mod_var['Email']) # Investment in Print and TV should be no more than the investment in Facebook and Email
    Mod.addConstr(Mod_var['Facebook']+ Mod_var['LinkedIn'] + Mod_var['Instagram'] + Mod_var['Snapchat'] + Mod_var['Twitter'] >= 2*(Mod_var['SEO'] + Mod_var['AdWords'])) # Total amount in social media shoule be atleaset twice in SEO and AdWords

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    globals()["Allocation_"+str(j+1)] = ModX.X
    globals()["Objective_Value_"+str(j+1)] = Mod.objVal

    for val in list(ModX.X):
        row.append(val)
    allocations.append(row)

allocations = pd.DataFrame(allocations, columns=roi_firms_all_cols)
allocations

,,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,Budget Allocation for ROI 0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,Budget Allocation for ROI 1,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


### Sensitivity Analysis

In [7]:
# Iterating over the ROIs received by the two independent firms, i.e, over "ROI_data.csv"
roi_firms = pd.read_csv(firm_rois_path)
roi_firms = roi_firms.drop(['Platform'], axis=1)
roi_firms_cols = list(roi_firms.columns)
roi_firms = np.array(roi_firms)
roi_firms_all_cols = ['']
for val in roi_firms_cols:
    roi_firms_all_cols.append(val)

initial_budget = 10 # Initialize the budget
upper_lower = []
correct_estimate = 0 # Correct roi if first is counted as zero
for j, rois in enumerate(roi_firms):
    if j==correct_estimate:
        Mod = gp.Model()
        n = len(rois)
        ModX = Mod.addMVar(n) # Initializing 10 variables for 10 marketing sources.

        # Creating a variable dictionary.
        Mod_var = {}
        for i in range(n):
            Mod_var[roi_firms_cols[i]] = ModX[i]
        
        # Creating the objective function.
        obj = 0
        for i in range(n):
            obj += (1 + rois[i])*ModX[i]
        
        Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

        # Adding Constraints
        # Constarint for total budget
        sum = 0
        for i in range(n):
            sum += ModX[i]
        Mod.addConstr(sum <= initial_budget) # Constarint for total budget
        Mod.addConstr(Mod_var['Print'] + Mod_var['TV'] <= Mod_var['Facebook'] + Mod_var['Email']) # Investment in Print and TV should be no more than the investment in Facebook and Email
        Mod.addConstr(Mod_var['Facebook']+ Mod_var['LinkedIn'] + Mod_var['Instagram'] + Mod_var['Snapchat'] + Mod_var['Twitter'] >= 2*(Mod_var['SEO'] + Mod_var['AdWords'])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
        for i in range(n): # Adding constraint for investment in each segment less than $ 3M
            Mod.addConstr(ModX[i] <= segment_budget)

        Mod.Params.OutputFlag = 0
        Mod.optimize()

        # Creatinfg a dataframe row for lower bound
        row = []
        row.append(f'Lower Bounds')
        for item in list(ModX.SAObjLow):
            row.append(item)
        upper_lower.append(row)

        # Creatinfg a dataframe row for upper bound
        row = []
        row.append(f'Lower Bounds')
        for item in list(ModX.SAObjUp):
            row.append(item)
        upper_lower.append(row)

upper_lower = pd.DataFrame(upper_lower, columns=roi_firms_all_cols)
upper_lower

,,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,Lower Bounds,-inf,1.039,-inf,1.033,-inf,-inf,1.039,-inf,-inf,1.029
1,Lower Bounds,1.049,1.062,1.039,1.046,1.029,1.039,inf,1.039,1.039,inf


### Stable budget analysis.

In [8]:
obj_vals = []
initial_budget = int(input("Enter the total marketing budget:")) # Initialize the budget
segment_budget = int(input("Enter the individual marketing budget upper bound:"))
obj_vals.append(initial_budget)
allocations = []
data_cols = list(monthly_roi.columns)[1:]
for i, month in enumerate(monthly_roi['Months']): # Iterating over each months ROI, assuming the initial investment to be $ 10M
    row = []
    row.append(month)
    rois = np.array(monthly_roi.drop('Months', axis=1))
    rois = rois[i]
    Mod = gp.Model()
    n = len(rois)
    ModX = Mod.addMVar(n) # Initializing 10 variables for 10 marketing sources.

    # Creating a variable dictionary.
    Mod_var = {}
    for m in range(n):
        Mod_var[data_cols[m]] = ModX[m]

    # Creating the objective function.
    obj = 0
    for m in range(n):
        obj += (1 + (rois[m]/100))*ModX[m]

    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    # Changing budget to reinvest 50% of the earnings each month.
    if i==0:
        budget = obj_vals[i]
    else:
        budget = budget + (obj_vals[i] - budget) / 2
    row.append(budget)

    # Adding Constraints
    # Constarint for total budget
    sum = 0
    for m in range(n):
        sum += ModX[m]
    Mod.addConstr(sum <= initial_budget) # Constarint for total budget
    Mod.addConstr(Mod_var['Print'] + Mod_var['TV'] <= Mod_var['Facebook'] + Mod_var['Email']) # Investment in Print and TV should be no more than the investment in Facebook and Email
    Mod.addConstr(Mod_var['Facebook']+ Mod_var['LinkedIn'] + Mod_var['Instagram'] + Mod_var['Snapchat'] + Mod_var['Twitter'] >= 2*(Mod_var['SEO'] + Mod_var['AdWords'])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
    for m in range(n): # Adding constraint for investment in each segment less than $ 3M
        Mod.addConstr(ModX[m] <= segment_budget)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    
    for x in ModX.X:
        row.append(x)
    row.append(Mod.objVal)
    obj_vals.append(Mod.objVal)
    allocations.append(row)

cols = ['Montths', 'Budget']
for val in data_cols:
    cols.append(val)

cols.append('Obj_Value')

allocations = pd.DataFrame(allocations, columns=cols)
allocations.to_csv('allocations.csv', index=False)
display(allocations)

,Montths,Budget,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Obj_Value
0,January,10.000000,3.0,0.0,0.0,1.333333,0.0,0.0,2.666667,0.0,0.000000,3.0,10.373000
1,February,10.186500,3.0,0.0,0.0,2.333333,3.0,0.0,0.000000,0.0,1.666667,0.0,10.399333
2,March,10.292917,0.0,0.0,0.0,3.000000,0.0,3.0,1.000000,0.0,3.000000,0.0,10.400000
3,April,10.346458,0.0,0.0,0.0,3.000000,0.0,3.0,3.000000,0.0,1.000000,0.0,10.393000
4,May,10.369729,1.0,0.0,0.0,0.000000,0.0,0.0,3.000000,0.0,3.000000,3.0,10.404000
5,June,10.386865,3.0,0.0,0.0,0.000000,0.0,0.0,3.000000,0.0,1.000000,3.0,10.416000
6,July,10.401432,0.0,0.0,0.0,3.000000,1.0,0.0,3.000000,0.0,3.000000,0.0,10.420000
7,August,10.410716,2.5,0.0,0.0,1.500000,0.0,0.0,0.000000,0.0,3.000000,3.0,10.429000
8,September,10.419858,0.5,0.0,0.0,3.000000,0.0,3.0,0.000000,0.0,3.000000,0.5,10.394500
9,October,10.407179,0.0,0.0,0.0,3.000000,0.0,3.0,3.000000,0.0,0.000000,1.0,10.365000
